# 共轭梯度法
一种解正定线性方程组的方法     
通过矩阵乘法（较简单的运算），实现矩阵求逆
在p步（A的维度）迭代之后可以得到精确解
其最大时间复杂度为p^3 和直接求解线性方程组的时间复杂度相同

### 1. CG 简单实现

解线性方程组 $Ax=b$，其中 $A$ 为正定矩阵。

In [1]:
import numpy as np
np.set_printoptions(linewidth=100)

# https://en.wikipedia.org/wiki/Conjugate_gradient_method
# A [m x m], b [m], x0 [m]
def cg(A, b, x0, eps=1e-3, print_progress=False):
    m = b.shape[0]
    # 初始解（注意此处应该复制x0，否则程序退出时会修改x0）
    x = np.copy(x0)
    # 初始残差向量
    r = b - np.dot(A, x)
    # 初始共轭梯度
    p = r

    for k in range(m):
        # 矩阵乘法
        Ap = np.dot(A, p)
        #这里存下来了 保证每次循环只做1次矩阵乘法
        #
        
        rr = r.dot(r)
        alpha = rr / p.dot(Ap)
        # 更新解
        x += alpha * p
        # 计算新残差向量
        rnew = r - alpha * Ap
        # 测试是否收敛
        norm = np.linalg.norm(rnew)
        if print_progress:
            print(f"Iter {k}, residual norm = {norm}")
        if norm < eps:
            break
        beta = rnew.dot(rnew) / rr
        # 更新共轭梯度
        p = rnew + beta * p
        # 更新残差向量
        r = rnew

    return x

进行简单的测试：

In [2]:
np.random.seed(123)
m = 10
x = np.random.normal(size=(m, m))
# A 为正定矩阵
A = x.transpose().dot(x)
b = np.random.normal(size=m)
# 直接求解
sol = np.linalg.solve(A, b)
sol

array([-136.71894832,  -33.85176812,   41.84149034,   73.42723076,   12.50311233,  -41.91028953,
         -6.01395804,  -30.7294723 ,  -24.21718752,  -68.80957079])

CG 求解：

In [3]:
cg(A, b, x0=np.zeros(shape=m), print_progress=True)

Iter 0, residual norm = 3.411450076020752
Iter 1, residual norm = 5.7628912322110075
Iter 2, residual norm = 7.818450458630144
Iter 3, residual norm = 7.264409878363998
Iter 4, residual norm = 10.319381541218602
Iter 5, residual norm = 8.114715301977098
Iter 6, residual norm = 4.326932882751054
Iter 7, residual norm = 6.726392470542996
Iter 8, residual norm = 9.429682571374672
Iter 9, residual norm = 4.527412132352139e-08


array([-136.71894832,  -33.85176812,   41.84149034,   73.42723076,   12.50311233,  -41.91028953,
         -6.01395804,  -30.7294723 ,  -24.21718752,  -68.80957079])

数学上可以证明，CG 可以保证在 $m$ 步后收敛，其中 $m$ 是 $A$ 的维度，但要注意前提 $A$ 正定。实际使用中如果 $A$ 的性质较好（最大特征值与最小特征值的比值（条件数）较小），CG 往往在远小于 $m$ 步时就可以收敛。

In [4]:
np.random.seed(123)
m = 1000
x = np.random.normal(size=(m, m))
# A 为正定矩阵
A = x.transpose().dot(x) / m
# 计算条件数，即最大特征值与最小特征值的比值
print(np.linalg.cond(A))
b = np.random.normal(size=m)
# 难以收敛
# cg(A, b, x0=np.zeros(shape=m), print_progress=True)
print()
A = A + np.eye(m)
print(np.linalg.cond(A))
sol = cg(A, b, x0=np.zeros(shape=m), print_progress=True)

12220940.931277275

5.024992685713433
Iter 0, residual norm = 15.524400175022853
Iter 1, residual norm = 6.275357557223779
Iter 2, residual norm = 2.444789104539785
Iter 3, residual norm = 0.9770907847185345
Iter 4, residual norm = 0.3761522298898335
Iter 5, residual norm = 0.14006062769910319
Iter 6, residual norm = 0.054044129768379495
Iter 7, residual norm = 0.020175188422398786
Iter 8, residual norm = 0.007640617406259871
Iter 9, residual norm = 0.002771863279343728
Iter 10, residual norm = 0.0010250641894911
Iter 11, residual norm = 0.00039961370358766827


因底层依靠矩阵乘法，故CG尤其适合矩阵乘法可高效运算的场合


但必须要保证正定性：
例如，

### 2. 更通用的 CG 实现

在上述实现中我们可以发现，用到 $A$ 的地方仅仅是计算矩阵乘法 $Ax$，而并未直接使用其他信息，例如 $A$ 中每个元素的取值。因此，我们可以传入一个计算矩阵乘法的函数给 CG，使其可以适用于不同类型的矩阵。

此处A由X和lambda所决定，A=X‘X+lambdaI     
X:n*p矩阵 只需存X和lambda 空间复杂度也降低      
则Av的时间复杂度则为n*p     
而若直接存储A 并直算Av 则在空间复杂度和时间复杂度上都较多      
Afn(x,X,lambda)

In [ ]:
def cg(Afn, b, x0, eps=1e-3, print_progress=False, **Afn_args):
    m = b.shape[0]
    # 初始解（注意此处应该复制x0，否则程序退出时会修改x0）
    x = np.copy(x0)
    # 初始残差向量
    r = b - Afn(x, **Afn_args)
    # 初始共轭梯度
    p = r

    for k in range(m):
        # 矩阵乘法
        Ap = Afn(p, **Afn_args)
        rr = r.dot(r)
        alpha = rr / p.dot(Ap)
        # 更新解
        x += alpha * p
        # 计算新残差向量
        rnew = r - alpha * Ap
        # 测试是否收敛
        norm = np.linalg.norm(rnew)
        if print_progress:
            print(f"Iter {k}, residual norm = {norm}")
        if norm < eps:
            break
        beta = rnew.dot(rnew) / rr
        # 更新共轭梯度
        p = rnew + beta * p
        # 更新残差向量
        r = rnew

    return x

此时要使用 CG 时我们需要提供一个函数，而不是 $A$ 本身：

In [ ]:
def mat_prod(x, mat):
    return mat.dot(x)

sol = cg(mat_prod, b, x0=np.zeros(shape=m), print_progress=True, mat=A)

这种通用写法的好处是我们可以针对一些特殊的矩阵定义高效的矩阵运算。以对角矩阵为例：

In [ ]:
def diag_mat_prod(x, diag_elements):
    return diag_elements * x  # 逐元素相乘，非矩阵乘法

sol = cg(diag_mat_prod, b, x0=np.zeros(shape=m), print_progress=True, diag_elements=np.diagonal(A))